<a href="https://colab.research.google.com/github/AryaSuryaaa/Pengembangan-Machine-Learning/blob/main/Multiclass_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # impor library pandas dan ubah dataset menjadi dataframe
import pandas as pd
# df = pd.read_csv('imdb_indonesian.csv')
# df = df.drop(columns=['judul_film'])
df = pd.read_csv('booksData.csv')
df = df.drop(columns=['title', 'index'])

In [ ]:
# Panggil fungsi head() pada dataframe untuk menampilkan 5 sampel teratas pada dataset.
df.head()

,genre,summary
0,fantasy,Drowned Wednesday is the first Trustee among ...
1,fantasy,"As the book opens, Jason awakens on a school ..."
2,fantasy,Cugel is easily persuaded by the merchant Fia...
3,fantasy,The book opens with Herald-Mage Vanyel return...
4,fantasy,Taran and Gurgi have returned to Caer Dallben...


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding


In [ ]:
# melakukan one-hot-encoding dan membuat dataframe baru.
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,summary,crime,fantasy,history,horror,psychology,romance,science,sports,thriller,travel
0,Drowned Wednesday is the first Trustee among ...,0,1,0,0,0,0,0,0,0,0
1,"As the book opens, Jason awakens on a school ...",0,1,0,0,0,0,0,0,0,0
2,Cugel is easily persuaded by the merchant Fia...,0,1,0,0,0,0,0,0,0,0
3,The book opens with Herald-Mage Vanyel return...,0,1,0,0,0,0,0,0,0,0
4,Taran and Gurgi have returned to Caer Dallben...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4652,"Atticus O’Sullivan, last of the Druids, lives ...",0,1,0,0,0,0,0,0,0,0
4653,Charlie Bucket's wonderful adventure begins wh...,0,1,0,0,0,0,0,0,0,0
4654,"""I live for the dream that my children will be...",0,1,0,0,0,0,0,0,0,0
4655,"Rose loves Dimitri, Dimitri might love Tasha, ...",0,1,0,0,0,0,0,0,0,0


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.


In [ ]:
# sinopsis = df_baru['ringkasan_sinopsis'].values
# label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values
sinopsis = df_baru['summary'].values
label = df_baru[['crime','fantasy','history','horror','psychology','romance','science','sports','thriller','travel']].values

In [ ]:
# mem bagi data untuk training dan data untuk testing
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

# mengonversi setiap sampel menjadi sequence
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih, padding='post', maxlen=5, truncating='post')
padded_test = pad_sequences(sekuens_test, padding='post', maxlen=5, truncating='post')

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# melatih model
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
117/117 - 4s - loss: 2.0727 - accuracy: 0.2185 - val_loss: 2.0598 - val_accuracy: 0.2092 - 4s/epoch - 32ms/step
Epoch 2/30
117/117 - 1s - loss: 1.9839 - accuracy: 0.2174 - val_loss: 2.0344 - val_accuracy: 0.2157 - 620ms/epoch - 5ms/step
Epoch 3/30
117/117 - 1s - loss: 1.8775 - accuracy: 0.2344 - val_loss: 2.0264 - val_accuracy: 0.2446 - 626ms/epoch - 5ms/step
Epoch 4/30
117/117 - 1s - loss: 1.7020 - accuracy: 0.3251 - val_loss: 2.1569 - val_accuracy: 0.2285 - 639ms/epoch - 5ms/step
Epoch 5/30
117/117 - 1s - loss: 1.5401 - accuracy: 0.4038 - val_loss: 2.2764 - val_accuracy: 0.2124 - 609ms/epoch - 5ms/step
Epoch 6/30
117/117 - 1s - loss: 1.4144 - accuracy: 0.4502 - val_loss: 2.4893 - val_accuracy: 0.2264 - 619ms/epoch - 5ms/step
Epoch 7/30
117/117 - 1s - loss: 1.3153 - accuracy: 0.4996 - val_loss: 2.6170 - val_accuracy: 0.2167 - 602ms/epoch - 5ms/step
Epoch 8/30
117/117 - 1s - loss: 1.2411 - accuracy: 0.5385 - val_loss: 2.8633 - val_accuracy: 0.2114 - 656ms/epoch - 6ms/step
Ep